In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
from utils import *
from display_rational import convert_res_to_htmls
from config import *
from losses import imbalanced_bce_bayesian, imbalanced_bce_resampling, exp_interval_loss
from metrices import *
from tqdm import tqdm_notebook
from bert import optimization
from bert import run_classifier
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.backend import set_session
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import os
import pickle
from collections import defaultdict
from datetime import datetime

import bert

import tensorflow
if tensorflow.__version__.startswith('2'):
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
else:
    import tensorflow as tf

Instructions for updating:
non-resource variables are not supported in the long term



In [3]:
# parse args
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--par_lambda', type=float)
parser.add_argument('--gpu_id', type=str)
parser.add_argument('--batch_size', type=int)
parser.add_argument('--num_epochs', type=int)
parser.add_argument('--dataset', type=str, choices='fever multirc movies'.split()) # fever, multirc, movie
parser.add_argument("--do_train", action='store_true')
parser.add_argument('--exp_visualize', action='store_true')
parser.add_argument('--evaluate', action='store_true')
parser.add_argument('--exp_benchmark', action='store_true')
parser.add_argument('--exp_structure', type=str, default='gru', choices='gru rnr'.split()) # gru, rnr
parser.add_argument('--delete_checkpoints', action='store_true')
parser.add_argument('--merge_evidences', action='store_true')
parser.add_argument('--benchmark_split', type=str, default='test', choices='test train val'.split()) # gru, rnr
parser.add_argument('--train_on_portion', type=float, default=0)

args = ['--par_lambda', '0.01', 
        '--gpu_id', '0', 
        '--batch_size', '2', 
        '--num_epochs', '10',
        '--dataset', 'movies',
        '--do_train',
        '--evaluate',
        '--exp_benchmark',
        '--exp_structure', 'rnr',
        '--delete_checkpoints',
        '--merge_evidences']

args = parser.parse_args(args)
args = parser.parse_args()

BATCH_SIZE = args.batch_size
par_lambda = args.par_lambda
NUM_EPOCHS = args.num_epochs
gpu_id = args.gpu_id
exp_structure = args.exp_structure
dataset = args.dataset
DO_DELETE = args.delete_checkpoints
do_train = args.do_train
load_best = not do_train
evaluate = args.evaluate
exp_visualize = args.exp_visualize
exp_benchmark = args.exp_benchmark
merge_evidences = args.merge_evidences
BENCHMARK_SPLIT_NAME = args.benchmark_split
train_on_portion = args.train_on_portion

LEARNING_RATE = 1e-5

In [4]:
import chakin

CHAKIN_INDEX = 13
NUMBER_OF_DIMENSIONS = 200
SUBFOLDER_NAME = "glove.6B"

DATA_FOLDER = "embeddings"
ZIP_FILE = os.path.join(DATA_FOLDER, "{}.zip".format(SUBFOLDER_NAME))
ZIP_FILE_ALT = "glove" + ZIP_FILE[5:]  # sometimes it's lowercase only...
UNZIP_FOLDER = os.path.join(DATA_FOLDER, SUBFOLDER_NAME)
if SUBFOLDER_NAME[-1] == "d":
    GLOVE_FILENAME = os.path.join(UNZIP_FOLDER, "{}.txt".format(SUBFOLDER_NAME))
else:
    GLOVE_FILENAME = os.path.join(UNZIP_FOLDER, "{}.{}d.txt".format(SUBFOLDER_NAME, NUMBER_OF_DIMENSIONS))

if not os.path.exists(ZIP_FILE) and not os.path.exists(UNZIP_FOLDER):
    # GloVe by Stanford is licensed Apache 2.0: 
    #     https://github.com/stanfordnlp/GloVe/blob/master/LICENSE
    #     http://nlp.stanford.edu/data/glove.twitter.27B.zip
    #     Copyright 2014 The Board of Trustees of The Leland Stanford Junior University
    print("Downloading embeddings to '{}'".format(ZIP_FILE))
    chakin.download(number=CHAKIN_INDEX, save_dir='./{}'.format(DATA_FOLDER))
else:
    print("Embeddings already downloaded.")
    
if not os.path.exists(UNZIP_FOLDER):
    import zipfile
    if not os.path.exists(ZIP_FILE) and os.path.exists(ZIP_FILE_ALT):
        ZIP_FILE = ZIP_FILE_ALT
    with zipfile.ZipFile(ZIP_FILE,"r") as zip_ref:
        print("Extracting embeddings to '{}'".format(UNZIP_FOLDER))
        zip_ref.extractall(UNZIP_FOLDER)
else:
    print("Embeddings already extracted.")

Embeddings already downloaded.
Embeddings already extracted.


In [ ]:
CACHE_DIR = 'cache/'
DATASET_NAME = f"lstm_attention_{dataset}_{'evidences_merged' if merge_evidences else 'evidences_splitted'}_train_on_portion_{train_on_portion}"
MODEL_NAME = DATASET_NAME + f'_par_lambda_{par_lambda}_lr_{LERANING_RATE}'
DATASET_CACHE = DATASET_NAME + '_inputdata_cache'
CHECKPOINTS_DIR = MODEL_NAME

if DO_DELETE:
    try:
        tf.gfile.DeleteRecursively(CHECKPOINTS_DIR)
    except:
        pass
if tensorflow.__version__.startswith('2'):
    tf.io.gfile.makedirs(CHECKPOINTS_DIR)
else:
    tf.gfile.MakeDirs(CHECKPOINTS_DIR)
print(f'***** Model output directory: {CHECKPOINTS_DIR} *****')

In [ ]:
import pandas as pd
pd.DataFrame({'docs':['this is good']})

In [5]:
import numpy as np
x = np.array([[1,2], [3,99]])
axis=-1
y = np.exp(x - np.max(x, axis, keepdims=True))
y / np.sum(y, axis, keepdims=True)


array([[2.68941421e-01, 7.31058579e-01],
       [2.03109266e-42, 1.00000000e+00]])

In [ ]:
# preprocessing
if dataset == 'movies':
    label_list = ['POS', 'NEG']
elif dataset == 'multirc' or dataset == 'boolq':
    label_list = ['True', 'False']
elif dataset == 'fever':
    label_list = ['SUPPORTS', 'REFUTES']
elif dataset == ''
        
from utils import cache_decorator
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_as_df_maybe_download(dataset_name, part):
    dataset_dir = f'~/.keras/dataset/{dataset_name}'
    from eraserbenchmark.rationale_benchmark.utils import load_datasets, load_documents
    
    data_dir = f'/home/zzhang/.keras/datasets/{dataset}/'
    train, val, test = load_datasets(data_dir)
if train_on_portion != 0:
    train = train[:int(len(train) * train_on_portion)]
    if not os.path.isdir(dataset_dir):
        import requests
        url = f'http://www.eraserbenchmark.com/zipped/{dataset_name}.tar.gz'
        target_path = f'{dataset_name}.tar.gz'
        response = requests.get(url, steam=True)
        if response.status_code == 200:
            os.mkdir(dataset_dir)
            with open()
        
        
        
        import requests

url = 'https://pypi.python.org/packages/source/x/xlrd/xlrd-0.9.4.tar.gz'
target_path = 'xlrd-0.9.4.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

@cache_decorator(os.path.join(CACHE_DIR, DATASET_NAME))
def data_preprocess_wrapper(dataset_name, train_on_portion):
    from copy import deepcopy
    docs_triple, queries_triple = [], []
    max_doc_len, max_query_len = 0, 0
    for part in 'train val test'.split():
        data = load_as_df_maybe_download(datset_name, part)
        docs_triple.append(tokenize(data[DOCS_COLUMN_NAME]))
        queries_triple.append(tokenize(data[QUERIES_COLUMN_NAME]))
        max_doc_len = max(max_doc_len, max(list(map(len, docs_triple[-1]))))
        max_query_len = max(max_query_len, max(list(map(len, queries_triple[-1]))))
    ret = []
    for docs, queries in zip(docs_triple, queries_triple):
        docs = pad_sequences(docs, maxlen=max_doc_len)
        queries = pad_sequences(queries, maxlen=max_query_len)
        ret.append((docs, queries))
    return ret, (max_doc_len, max_query_len)
    
(train, val, test), (max_doc_len, max_query_len) =\
    data_preprocess_wrapper(dataset, train_on_portion=train_on_portion)

input_data = padding(data)

In [5]:
# load embeddings
def load_embedding_from_disks(glove_filename, with_indexes=True):
    """
    Read a GloVe txt file. If `with_indexes=True`, we return a tuple of two dictionnaries
    `(word_to_index_dict, index_to_embedding_array)`, otherwise we return only a direct 
    `word_to_embedding_dict` dictionnary mapping from a string to a numpy array.
    """
    if with_indexes:
        word_to_index_dict = dict()
        index_to_embedding_array = []
    else:
        word_to_embedding_dict = dict()

    with open(glove_filename, 'r') as glove_file:
        for (i, line) in enumerate(glove_file):
            split = line.split(' ')
            word = split[0]
            representation = split[1:]
            representation = np.array([float(val) for val in representation])
            if with_indexes:
                word_to_index_dict[word] = i
                index_to_embedding_array.append(representation)
            else:
                word_to_embedding_dict[word] = representation

    _WORD_NOT_FOUND = [0.0]* len(representation)  # Empty representation for unknown words.
    if with_indexes:
        _LAST_INDEX = i + 1
        word_to_index_dict = defaultdict(lambda: _LAST_INDEX, word_to_index_dict)
        index_to_embedding_array = np.array(index_to_embedding_array + [_WORD_NOT_FOUND])
        return word_to_index_dict, index_to_embedding_array
    else:
        word_to_embedding_dict = defaultdict(lambda: _WORD_NOT_FOUND)
        return word_to_embedding_dict
    
print("Loading embedding from disks...")
word_to_index, index_to_embedding = load_embedding_from_disks(GLOVE_FILENAME, with_indexes=True)
print("Embedding loaded from disks.")

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Bidirectional, Attention, Dense, Concatenate, Conv2D
# from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import CuDNNGRU as GRU

GRU_DIM = 200


doc_input = Input(shape=(MAX_DOC_LEN, ), name='doc_input')
doc_mask_input = Input(shape=(MAX_DOC_LEN, ), name='doc_mask_input')

query_input = Input(shape=(MAX_QUERY_LEN, ), name='query_input')
query_mask_input = Input(shape=(MAX_QUERY_LEN, ), name='query_mask_input')

embedding_layer = Embedding(vocab_size, embedding_dim,
                            trainable=False, mask_zero=True, weights=[index_to_embedding])

pre_H = embedding_layer(doc_input)
pre_U = embedding_layer(query_input)

H = Dropout(rate=0.2)(Bidirectional(layer=GRU(GRU_DIM, return_sequence=True),
                  merge_mode='concat')(pre_H))  # Batch * T * 2d
U = Dropout(rate=0.2)(Bidirectional(layer=GRU(GRU_DIM, return_sequence=True),
                  merge_mode='concat')(pre_U))  # Batch * J * 2d

H_expanded = Lambda(lambda x: tf.tile(
    tf.expand_dims(x, 2), (1, 1, MAX_QUERY_LEN, 1)))(H)
U_expanded = Lambda(lambda x: tf.tile(
    tf.expand_dims(x, 1), (1, MAX_DOC_LEN, 1, 1)))(U)

HUH = Concatenate(axis=-1)([H_expanded, U_expanded, H_expanded * U_expanded])

Stj = Lambda(lambda x: tf.squeeze(x))(Conv2D(
    1, (1, 1), use_bias=False, data_format='channel_last')(HUH))  # Batch * T * J

U_tilde = Lambda(lambda x, y: tf.matmul(tf.nn.softmax(x, axis=2), y))(Stj, U) # T * 2d
H_tilde = Lambda(lambda x, y: 
                 tf.transpose(
                     tf.tile(
                         tf.matmul(
                             tf.transpose(
                                 tf.nn.softmax(
                                     tf.math.reduce_max(
                                         x, 
                                         axis=-1,
                                         keepdims=True
                                     ), 
                                     axis=-2
                                 ), 
                                 perm=(0, 2, 1)
                             ),
                             y
                         ), 
                         (1, MAX_DOC_LEN, 1)
                     ),
                     perm=(0, 2, 1)
                 )(Stj, H) # Batch * T * 2d

                 
G = Concatenate(axis=-1)([H, U_tilde, H * U_tilde, H * H_tilde])

M1=Bidirectional(layer=CuDNNLSTM(NUM_INTERVAL_LSTM_WIDTH, return_sequences=True),
                   merge_mode='concat')(gru2_seq)
p_starts=Dense(1, activation='sigmoid')(
    Concatenate(axis=-1)([concat_attention, M1]))

m1_tilde=Dot(axes=-2)([p_starts, M1])
M1_tilde=Lambda(lambda x: tf.tile(x, (1, MAX_SEQ_LENGTH, 1)))(m1_tilde)
x=Multiply()([M1, M1_tilde])
M2=Bidirectional(layer=CuDNNLSTM(NUM_INTERVAL_LSTM_WIDTH, return_sequences=True),
                   merge_mode='concat')(Concatenate(axis=-1)([concat_attention, M1, M1_tilde, x]))
p_end_given_start=Dense(MAX_SEQ_LENGTH, activation='softmax')(
    Concatenate(axis=-1)([concat_attention, M2]))
p_end_given_start=Lambda(
    lambda x: tf.linalg.band_part(x, 0, -1))(p_end_given_start)
exp_outputs=Concatenate(
    axis=-1, name='exp_output')([p_starts, p_end_given_start])
outputs.append(exp_outputs)

if par_lambda is None:
    loss_weights=None
else:
    loss_weights={'cls_output': 1,
                    'exp_output': par_lambda}
metrics={'cls_output': 'accuracy',
           'exp_output': [f1_wrapper(EXP_OUTPUT),
                          sp_precision_wrapper(EXP_OUTPUT),
                          sp_recall_wrapper(EXP_OUTPUT),
                          precision_wrapper(EXP_OUTPUT),
                          recall_wrapper(EXP_OUTPUT)]}
loss={'cls_output': 'binary_crossentropy',
        'exp_output': loss_function()}